In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/sound_depression.db'
mask_file = '/home/eli/Documents/sound_depression/fmriprep/all_masks/wholebrain.nii.gz'

OVERWRITE = True

In [4]:
fmriprep_dir = '/home/eli/Documents/sound_depression/fmriprep/'

In [5]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file, smooth=6)

if not existed or OVERWRITE:
    data_files = glob.glob('/home/eli/Documents/sound_depression/fmriprep/sub-*/func/*_preproc.nii.gz')
    regex = re.compile("sub-([A-Za-z]+)([0-9]{1,2})_task-([A-Za-z]+)_run-([0-9]{1,2})_\S+preproc.nii.gz")
    for data_file in data_files:
        logging.info("Recording blocks in %s", data_file)
        m = regex.search(data_file)
        condition, sub_num, task_set, run = m.groups()
        subject = condition + sub_num
        run = int(run)
        if run is None:
            run = ''
        tsv_file = '/sub-%s_task-%s_run-%01d_events.tsv' % (subject, task_set, run)
        tsv_path = '/home/eli/Documents/sound_depression/ds000171-download/sub-%s/func/' % subject
        tsv_file = os.path.dirname(tsv_path) + tsv_file
        with open(tsv_file, 'r') as tsv:
            tsv_reader = csv.DictReader(tsv, dialect='excel-tab')
            for row in tsv_reader:
                onset, duration, task = row.values()
                onset = round(float(onset) / 3)
                duration = round(float(duration) / 3)
                task_block = niidb.FMriActivationBlock(zscore=True)
                task_block.task = task
                task_block.filename = data_file
                task_block.subject = subject
                task_block.start_time = onset
                task_block.end_time = onset + duration
                task_block.individual_differences = {'condition': condition}
                db.upsert(task_block)
                logging.info('Stimulus %s from %d to %d', task, onset, onset + duration)

01/13/2019 19:19:07 Recording blocks in /home/eli/Documents/sound_depression/fmriprep/sub-mdd07/func/sub-mdd07_task-music_run-2_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
01/13/2019 19:19:07 Stimulus tones from 0 to 11
01/13/2019 19:19:07 Stimulus response from 11 to 12
01/13/2019 19:19:07 Stimulus positive_music from 12 to 22
01/13/2019 19:19:07 Stimulus response from 22 to 23
01/13/2019 19:19:07 Stimulus tones from 24 to 34
01/13/2019 19:19:07 Stimulus response from 34 to 35
01/13/2019 19:19:08 Stimulus negative_music from 35 to 45
01/13/2019 19:19:08 Stimulus response from 46 to 47
01/13/2019 19:19:08 Stimulus tones from 46 to 56
01/13/2019 19:19:08 Stimulus response from 57 to 58
01/13/2019 19:19:08 Stimulus positive_music from 58 to 68
01/13/2019 19:19:08 Stimulus response from 68 to 69
01/13/2019 19:19:08 Stimulus tones from 70 to 80
01/13/2019 19:19:08 Stimulus response from 80 to 81
01/13/2019 19:19:08 Stimulus negative_music from 81 to 91
01/13/2019 19:19:08 Stimulus respon

In [6]:
logging.info('Finished building NiiDb out of Lepping 2017 dataset')

01/13/2019 19:22:27 Finished building NiiDb out of Lepping 2017 dataset
